# Getting started with flex flow


**Learning Objectives** - Upon completing this tutorial, you should be able to:

- Write LLM application using notebook and visualize the trace of your application.
- Convert the application into a flow and batch run against multi lines of data.


## 0. Install dependent packages

In [3]:
%%capture --no-stderr
%pip install -r ./requirements.txt

## 1. Trace your application with promptflow

Assume we already have a python function that calls Unify AI. 

In [4]:
with open("llm.py") as fin:
    print(fin.read())

import os

from dotenv import load_dotenv
from openai.version import VERSION as OPENAI_VERSION

from promptflow.tracing import trace


def get_client():
    if OPENAI_VERSION.startswith("0."):
        raise Exception(
            "Please upgrade your OpenAI package to version >= 1.0.0 or using the command: pip install --upgrade openai."
        )
    api_key = os.environ.get("OPENAI_API_KEY", None)
    if api_key:
        from openai import OpenAI

        return OpenAI()
    else:
        from openai import AzureOpenAI

        return AzureOpenAI(
            api_version=os.environ.get("OPENAI_API_VERSION", "2023-07-01-preview")
        )


@trace
def my_llm_tool(
    prompt: str,
    # for AOAI, deployment name is customized by user, not model name.
    deployment_name: str,
    max_tokens: int = 120,
    temperature: float = 1.0,
    top_p: float = 1.0,
    n: int = 1,
) -> str:
    if "OPENAI_API_KEY" not in os.environ and "AZURE_OPENAI_API_KEY" not in os.environ:
        # load en

Note: before running below cell, please configure required environment variable `UNIFY_API_KEY` by create an `.env` file. Please refer to `./.env.example` as an template.

In [1]:
# Define provider and model from unify. Refer to https://unify.ai/benchmarks
model_name = "llama-3.1-8b-chat"
provider_name = "octoai"

In [2]:

from llm import my_llm_tool

# pls configure `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_ENDPOINT` environment variables first
result = my_llm_tool(
    prompt="Write a simple Hello, world! python program that displays the greeting message when executed. Output code only.",
    model_name=model_name,
    provider_name=provider_name,
)
result

'```python\nprint("Hello, world!")\n```'

### Visualize trace by using start_trace

Note we add `@trace` in the `my_llm_tool` function, re-run below cell will collect a trace in trace UI.

In [3]:
from promptflow.tracing import start_trace

# start a trace session, and print a url for user to check trace
start_trace()
# rerun the function, which will be recorded in the trace
result = my_llm_tool(
    prompt="Write a simple Hello, world! program that displays the greeting message when executed. Output code only.",
    model_name=model_name,
    provider_name=provider_name,
)
result

Prompt flow service has started...


'```cpp\n#include <iostream>\n\nint main() {\n    std::cout << "Hello, world!" << std::endl;\n    return 0;\n}\n```'

You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=unify-llm-tool&uiTraceId=0x8b3ca5c573696019dd080376199ba71f
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=unify-llm-tool&uiTraceId=0xfe5c73bc84f9df87d8a792c6b7ae496b


Now, let's add another layer of function call. In `programmer.py` there is a function called `write_simple_program`, which calls a new function called `load_prompt` and previous `my_llm_tool` function.

In [4]:
# show the programmer.py content
with open("programmer.py") as fin:
    print(fin.read())

from pathlib import Path
from typing import TypedDict

from jinja2 import Template
from llm import my_llm_tool

from promptflow.tracing import trace

BASE_DIR = Path(__file__).absolute().parent


class Result(TypedDict):
    output: str


@trace
def load_prompt(jinja2_template: str, text: str) -> str:
    """Load prompt function."""
    with open(BASE_DIR / jinja2_template, "r", encoding="utf-8") as f:
        prompt = Template(
            f.read(), trim_blocks=True, keep_trailing_newline=True
        ).render(text=text)
        return prompt


@trace
def write_simple_program(
    text: str = "Hello World!", model_name="gpt-4", provider_name="openai"
) -> Result:
    """Ask LLM to write a simple program."""
    prompt = load_prompt("hello.jinja2", text)
    output = my_llm_tool(prompt=prompt, model_name=model_name, provider_name=provider_name, max_tokens=120)
    return Result(output=output)


if __name__ == "__main__":
    from promptflow.tracing import start_trace

    start_trace()

In [5]:
# call the flow entry function
from programmer import write_simple_program

result = write_simple_program("Java Hello, world!")
result

{'output': 'public class HelloWorld {\n    public static void main(String[] args) {\n        System.out.println("Hello, world!");\n    }\n}'}

### Setup model configuration with environment variables

When used in local, create a model configuration object with environment variables.

In [16]:
import os
from dotenv import load_dotenv

from promptflow.core import AzureOpenAIModelConfiguration

if "AZURE_OPENAI_API_KEY" not in os.environ:
    # load environment variables from .env file
    load_dotenv()

if "AZURE_OPENAI_API_KEY" not in os.environ:
    raise Exception("Please specify environment variables: AZURE_OPENAI_API_KEY")
model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_deployment=deployment_name,
    api_version="2023-07-01-preview",
)

### Eval the result 

In [17]:
%load_ext autoreload
%autoreload 2

import paths  # add the code_quality module to the path
from code_quality import CodeEvaluator

evaluator = CodeEvaluator(model_config=model_config)
eval_result = evaluator(result)
eval_result

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[2024-07-28 11:08:34 +0000][promptflow.core._prompty_utils][ERROR] - Exception occurs: RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2023-07-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 4 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}
[2024-07-28 11:08:34 +0000][promptflow.core._prompty_utils][WARNING] - RateLimitError #0, Retry-After=4, Back off 4.0 seconds for retry.
[2024-07-28 11:08:38 +0000][promptflow.core._prompty_utils][ERROR] - Exception occurs: RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2023-07-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 6 seconds. Please go here: https://aka.ms/o

{'correctness': 5,
 'readability': 4,
 'explanation': "The code is correct as it successfully prints 'Hello, world!' in Java. The readability is slightly reduced due to the JSON format wrapping the code, but the Java code itself is well-formatted and easy to understand."}

## 2. Batch run the function as flow with multi-line data

Create a [flow.flex.yaml](https://github.com/microsoft/promptflow/blob/main/examples/flex-flows/basic/flow.flex.yaml) file to define a flow which entry pointing to the python function we defined.


In [ ]:
# show the flow.flex.yaml content
with open("flow.flex.yaml") as fin:
    print(fin.read())

### Batch run with a data file (with multiple lines of test data)


In [ ]:
from promptflow.client import PFClient

pf = PFClient()

In [ ]:
data = "./data.jsonl"  # path to the data file
# create run with the flow function and data
base_run = pf.run(
    flow=write_simple_program,
    data=data,
    column_mapping={
        "text": "${data.text}",
    },
    stream=True,
)

In [ ]:
details = pf.get_details(base_run)
details.head(10)

## 3. Evaluate your flow
Then you can use an evaluation method to evaluate your flow. The evaluation methods are also flows which usually using LLM assert the produced output matches certain expectation. 

### Run evaluation on the previous batch run
The **base_run** is the batch run we completed in step 2 above, for web-classification flow with "data.jsonl" as input.

In [ ]:
# we can also run flow pointing to yaml file
eval_flow = "../eval-code-quality/flow.flex.yaml"

eval_run = pf.run(
    flow=eval_flow,
    init={"model_config": model_config},
    data="./data.jsonl",  # path to the data file
    run=base_run,  # specify base_run as the run you want to evaluate
    column_mapping={
        "code": "${run.outputs.output}",
    },
    stream=True,
)

In [ ]:
details = pf.get_details(eval_run)
details.head(10)

In [ ]:
import json

metrics = pf.get_metrics(eval_run)
print(json.dumps(metrics, indent=4))

In [ ]:
pf.visualize([base_run, eval_run])

## Next steps

By now you've successfully run your first prompt flow and even did evaluation on it. That's great!

You can check out more examples:
- [Basic Chat](https://github.com/microsoft/promptflow/tree/main/examples/flex-flows/chat-basic): demonstrates how to create a chatbot that can remember previous interactions and use the conversation history to generate next message.